GRUPO GANADOR- PARTIDO 3

PRIMER INFORME

In [1]:
from itertools import count
from statistics import geometric_mean
import pandas as pd
import numpy as np

Abrimos las bases de datos "resultados_paso" y "circuitos_electorales"(una base descargada del GCBA que usaremos para vincular circuitos electorales con comunas)

In [4]:
resultados_paso = pd.read_csv(
    "paso_x_partido.csv" "",
    delimiter=",",  # delimitador ',',';','|','\t'
    header=0,  # número de fila como nombre de columna
    names=None,  # nombre de las columnas (ojo con header)
    index_col=0,  # que col es el índice
    usecols=None,  # que col usar. Ej: [0, 1, 2], ['foo', 'bar', 'baz']
    dtype=None,  # Tipo de col {'a': np.int32, 'b': str}
    skiprows=None,  # saltear filas al inicio
    skipfooter=0,  # saltear filas al final
    nrows=None,  # n de filas a leer
    decimal=".",  # separador de decimal. Ej: ',' para EU dat
    quotechar='"',  # char para reconocer str
    encoding=None,
)

resultados_paso  # archivos con tilde y ñ por lo general utilizan "utf-8" etc
#%%
comunas = pd.read_csv(
    "circuitos-electorales.csv",
    delimiter=",",  # delimitador ',',';','|','\t'
    header=0,  # número de fila como nombre de columna
    names=None,  # nombre de las columnas (ojo con header)
    index_col=0,  # que col es el índice
    usecols=None,  # que col usar. Ej: [0, 1, 2], ['foo', 'bar', 'baz']
    dtype=None,  # Tipo de col {'a': np.int32, 'b': str}
    skiprows=None,  # saltear filas al inicio
    skipfooter=0,  # saltear filas al final
    nrows=None,  # n de filas a leer
    decimal=".",  # separador de decimal. Ej: ',' para EU dat
    quotechar='"',  # char para reconocer str
    encoding=None,
)

vinculamos circuitos electorales con comunas

In [5]:
circuito_comuna = comunas[["COMUNA", "CIRCUITO_N", "BARRIO"]].rename(
    columns={"CIRCUITO_N": "circuito"}
)
circuito_comuna = circuito_comuna.sort_values(by=["circuito"])
circuito_comuna = circuito_comuna.reset_index(drop=True)

tabla_final = pd.merge(
    resultados_paso, circuito_comuna, on="circuito", how="outer", indicator=True
)
tabla_final

,circuito,urna,pp1,pp2,pp3,pp4,nv,COMUNA,BARRIO,_merge
0,1,0,64,74,49,50,63,1,SAN TELMO,both
1,1,1,79,81,44,36,60,1,SAN TELMO,both
2,1,2,72,74,55,34,65,1,SAN TELMO,both
3,1,3,82,80,37,38,63,1,SAN TELMO,both
4,1,4,64,87,57,28,64,1,SAN TELMO,both
...,...,...,...,...,...,...,...,...,...,...
7634,167,32,76,46,56,66,56,15,AGRONOMIA,both
7635,167,33,62,59,51,57,71,15,AGRONOMIA,both
7636,167,34,63,52,51,69,65,15,AGRONOMIA,both
7637,167,35,68,66,51,64,51,15,AGRONOMIA,both


DISTRIBUCION DE LOS VOTOS DE CADA CANDIDATO POR COMUNA

In [6]:
distrib_candidatos = (
    tabla_final[["COMUNA", "pp1", "pp2", "pp3", "pp4", "nv", "BARRIO"]]
    .groupby(["COMUNA"])
    .sum()
    .transform(lambda x: (x + 0.0) / x.sum() * 100)
)
distrib_candidatos


,pp1,pp2,pp3,pp4,nv
COMUNA,,,,,
1,7.232794,9.450333,5.152216,4.998396,6.698759
2,4.439748,4.805121,7.125770,6.884241,5.815161
3,7.573100,7.902294,5.214219,5.623694,6.537122
4,8.411547,9.968829,4.995669,5.114962,7.105827
5,6.772836,6.259406,6.069294,6.360912,6.419809
6,5.903311,4.896027,7.384997,7.114055,6.272286
7,7.524485,8.139089,7.119394,6.815452,7.480038
8,6.617667,9.496991,3.263092,3.665963,5.781864
9,6.131736,5.339826,5.227412,5.180434,5.415370


TOTAL DE VOTOS DE CADA CANDIDATO X COMUNA

In [8]:
total_votos_x_comunas = (
    tabla_final[["COMUNA", "pp1", "pp2", "pp3", "pp4", "nv", "BARRIO"]]
    .groupby(["COMUNA"])
    .sum()
)
total_votos_x_comunas

,pp1,pp2,pp3,pp4,nv
COMUNA,,,,,
1,32582,43142,23433,22598,30378
2,20000,21936,32409,31124,26371
3,34115,36075,23715,25425,29645
4,37892,45509,22721,23125,32224
5,30510,28575,27604,28758,29113
6,26593,22351,33588,32163,28444
7,33896,37156,32380,30813,33921
8,29811,43355,14841,16574,26220
9,27622,24377,23775,23421,24558


creamos una columna para ver la candiad de votantes de cada comuna

In [9]:
total_votos_x_comunas = total_votos_x_comunas.assign(
    total_x_comuna=lambda x: (x["pp1"] + x["pp2"] + x["pp3"] + x["pp4"] + x["nv"])
)
total_votos_x_comunas

,pp1,pp2,pp3,pp4,nv,total_x_comuna
COMUNA,,,,,,
1,32582,43142,23433,22598,30378,152133
2,20000,21936,32409,31124,26371,131840
3,34115,36075,23715,25425,29645,148975
4,37892,45509,22721,23125,32224,161471
5,30510,28575,27604,28758,29113,144560
6,26593,22351,33588,32163,28444,143139
7,33896,37156,32380,30813,33921,168166
8,29811,43355,14841,16574,26220,130801
9,27622,24377,23775,23421,24558,123753


Ahora calculamos que porcentaje de votos obtuvo cada candidato en cada comuna

In [10]:
porcentaje_votos_x_comunas = total_votos_x_comunas.transform(
    (lambda x: (x + 0.0) / x["total_x_comuna"] * 100), axis=1
)
porcentaje_votos_x_comunas

,pp1,pp2,pp3,pp4,nv,total_x_comuna
COMUNA,,,,,,
1,21.416787,28.358081,15.402970,14.854108,19.968054,100.0
2,15.169903,16.638350,24.582069,23.607403,20.002275,100.0
3,22.899815,24.215472,15.918778,17.066622,19.899312,100.0
4,23.466753,28.184008,14.071257,14.321457,19.956525,100.0
5,21.105423,19.766879,19.095185,19.893470,20.139043,100.0
6,18.578445,15.614892,23.465303,22.469767,19.871593,100.0
7,20.156274,22.094835,19.254784,18.322967,20.171140,100.0
8,22.791110,33.145771,11.346244,12.671157,20.045718,100.0
9,22.320267,19.698108,19.211655,18.925602,19.844367,100.0


Ordenamos las comunas en función del desempeño de nuestro candidato

In [11]:
porcentaje_orden = porcentaje_votos_x_comunas.sort_values(["pp3"], ascending=False)
porcentaje_orden

,pp1,pp2,pp3,pp4,nv,total_x_comuna
COMUNA,,,,,,
13,16.604400,14.295679,25.286014,23.722911,20.090996,100.0
2,15.169903,16.638350,24.582069,23.607403,20.002275,100.0
14,16.078886,16.636939,23.760801,23.658268,19.865106,100.0
6,18.578445,15.614892,23.465303,22.469767,19.871593,100.0
11,19.474565,14.761044,23.433066,22.445114,19.886212,100.0
12,18.985773,15.623074,22.278741,23.106808,20.005603,100.0
10,20.415646,16.331139,21.742192,21.344917,20.166105,100.0
15,20.400949,18.808844,19.811561,20.891991,20.086655,100.0
7,20.156274,22.094835,19.254784,18.322967,20.171140,100.0


Abrimos las bases de datos del censo

In [22]:
hogar = pd.read_csv("censo/hogar.csv", sep=",")

#
vivienda = pd.read_csv("censo/vivienda.csv", sep=",")

#
persona = pd.read_csv("censo/persona.csv", sep=",")

#
prov = pd.read_csv("censo/prov.csv", sep=",")

# 
radio = pd.read_csv("censo/radio.csv")

# 
frac = pd.read_csv("censo/frac.csv")


utilizamos la siguiente linea de codigo para vincular las comunas, con las personas, viviendas y hogares

In [18]:
dpto = pd.read_csv("censo/dpto.csv")
dpto = dpto[["DPTO_REF_ID", "NOMDPTO"]]
#dpto
# %%
dpto_frac = pd.merge(dpto, frac, on="DPTO_REF_ID", how="left")
dpto_frac.groupby = ["NOMDPTO"]
#dpto_frac
# %%
dpto_frac_radio = pd.merge(dpto_frac, radio, on="FRAC_REF_ID", how="right")
#dpto_frac_radio

# %%
dpto_frac_radio_vivienda = pd.merge(
    dpto_frac_radio, vivienda, on="RADIO_REF_ID", how="right", indicator=True
)
dpto_frac_radio_vivienda.rename(columns={"DPTO_REF_ID": "COMUNA"}, inplace=True)
#dpto_frac_radio_vivienda
#%%
# dpto_frac_radio_vivienda[dpto_frac_radio_vivienda.columns[[0,6,7,8,9,10,11,12,13,14,15,16,17]]]
dpto_frac_radio_vivienda = dpto_frac_radio_vivienda[
    dpto_frac_radio_vivienda.columns[[0, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
].copy()
# %%
#dpto_frac_radio_vivienda
# %%
hogarSUPREMO = pd.merge(
    dpto_frac_radio_vivienda, hogar, on="VIVIENDA_REF_ID", how="right"
)

en la siguiente variable, se almacena por comuna el cruce de todas las variables de hogares y vivienda

In [19]:
hogarSUPREMO

,COMUNA,VIVIENDA_REF_ID,TIPVV,V01,V02,V00,URP,INCALSERV,INMAT,MUNI,...,H15,H16,H19A,H19B,H19C,H19D,PROP,INDHAC,TOTPERS,ALGUNBI
0,1,1,1,1,1,0,1,1,1,20010001,...,2,2,1,2,1,2,5,1,1,0
1,1,1,1,1,1,0,1,1,1,20010001,...,1,2,1,1,1,1,1,1,1,0
2,1,1,1,1,1,0,1,1,1,20010001,...,3,5,1,1,1,1,1,1,2,0
3,1,1,1,1,1,0,1,1,1,20010001,...,3,5,1,1,1,1,5,6,8,1
4,1,2,1,7,1,0,1,1,2,20010001,...,5,5,1,1,2,1,6,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152594,15,1426434,1,4,1,0,1,1,1,20150001,...,2,3,1,1,1,1,1,1,1,0
1152595,15,1426435,1,4,1,0,1,1,1,20150001,...,1,2,1,2,2,1,1,1,1,0
1152596,15,1426436,1,4,1,0,1,1,1,20150001,...,2,3,1,1,1,1,1,3,3,0
1152597,15,1426437,2,0,0,9,1,0,0,20150001,...,0,0,0,0,0,0,0,7,0,2


en la siguiente variable, se almacena por comuna el cruce de todas las variables de hogares, viviendas y personas

In [20]:
personaSUPREMO = pd.merge(hogarSUPREMO, persona, on="HOGAR_REF_ID", how="right")
personaSUPREMO

,COMUNA,VIVIENDA_REF_ID,TIPVV,V01,V02,V00,URP,INCALSERV,INMAT,MUNI,...,P05,P06,P07,P12,EDADAGRU,EDADQUI,P08,P09,P10,CONDACT
0,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,2,2,13,2,4,2,1
1,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,2,10,2,4,1,1
2,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,3,14,2,7,2,1
3,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,2,13,2,6,1,3
4,1,1,1,1,1,0,1,1,1,20010001,...,2,221,1,1,2,5,1,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890146,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,20,2,2,1,0
2890147,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,19,2,6,1,0
2890148,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,17,2,2,1,0
2890149,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,18,2,2,1,0


A continuacíon, nos preguntamos sobre cuantas personas del censo son votantes

In [21]:
votantes = personaSUPREMO[personaSUPREMO["P03"] >= 18]
votantes

,COMUNA,VIVIENDA_REF_ID,TIPVV,V01,V02,V00,URP,INCALSERV,INMAT,MUNI,...,P05,P06,P07,P12,EDADAGRU,EDADQUI,P08,P09,P10,CONDACT
0,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,2,2,13,2,4,2,1
1,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,2,10,2,4,1,1
2,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,3,14,2,7,2,1
3,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,2,13,2,6,1,3
4,1,1,1,1,1,0,1,1,1,20010001,...,2,221,1,1,2,5,1,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890146,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,20,2,2,1,0
2890147,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,19,2,6,1,0
2890148,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,17,2,2,1,0
2890149,15,1426438,2,0,0,1,1,0,0,20150001,...,1,0,1,0,3,18,2,2,1,0


Procedimiento para el analisis comuna de un primer grupo de variables significativas

tomamos la comuna 1 para graficar este analisis pormenorizado

Filtramos para obtener solo los casos de la comuna 1

In [25]:
is_comuna1 = votantes.loc[:,"COMUNA"] == 1
comuna1 = votantes[is_comuna1]
comuna1

,COMUNA,VIVIENDA_REF_ID,TIPVV,V01,V02,V00,URP,INCALSERV,INMAT,MUNI,...,P05,P06,P07,P12,EDADAGRU,EDADQUI,P08,P09,P10,CONDACT
0,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,2,2,13,2,4,2,1
1,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,2,10,2,4,1,1
2,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,3,14,2,7,2,1
3,1,1,1,1,1,0,1,1,1,20010001,...,1,0,1,1,2,13,2,6,1,3
4,1,1,1,1,1,0,1,1,1,20010001,...,2,221,1,1,2,5,1,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205881,1,131382,2,0,0,1,1,0,0,20010001,...,1,0,1,0,3,15,2,4,2,0
205882,1,131382,2,0,0,1,1,0,0,20010001,...,1,0,1,0,3,19,2,2,1,0
205883,1,131382,2,0,0,1,1,0,0,20010001,...,1,0,1,0,3,17,2,4,1,0
205884,1,131382,2,0,0,1,1,0,0,20010001,...,1,0,1,0,3,18,2,2,1,0


VARIABLES DE POBLACION

creamos un dataframe que nos muestra las variables de población para la comuna 1

In [27]:
comuna1_poblacion = comuna1[["COMUNA", "P03", "P05", "P07", "P08", "P09", "CONDACT"]].copy()
comuna1_poblacion

,COMUNA,P03,P05,P07,P08,P09,CONDACT
0,1,61,1,1,2,4,1
1,1,48,1,1,2,4,1
2,1,65,1,1,2,7,1
3,1,62,1,1,2,6,3
4,1,20,2,1,1,4,1
...,...,...,...,...,...,...,...
205881,1,72,1,1,2,4,0
205882,1,90,1,1,2,2,0
205883,1,84,1,1,2,4,0
205884,1,87,1,1,2,2,0


Medidas estadisticas por edad en la comuna

In [28]:
edad_c1 = comuna1_poblacion["P03"].describe()
edad_c1

count    168197.000000
mean         44.406523
std          18.825965
min          18.000000
25%          28.000000
50%          41.000000
75%          58.000000
max         110.000000
Name: P03, dtype: float64

moda del ultimo nivel educativo alcanzado

In [29]:
moda_educacion_c1 = comuna1_poblacion["P09"].mode()
moda_educacion_c1

0    4
Name: P09, dtype: int64

In [ ]:
Cantidad de casos por ultimo nivel educativo alcanzado

educacion_c1 = comuna1_poblacion.groupby(['P09']).count()
educacion_c1

porcentaje del ultimo nivel educativo alcanzado

In [31]:
porcentaje_educacion_c1 = educacion_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_educacion_c1

,COMUNA,P03,P05,P07,P08,CONDACT
P09,,,,,,
0,1.016070,1.016070,1.016070,1.016070,1.016070,1.016070
1,0.054698,0.054698,0.054698,0.054698,0.054698,0.054698
2,16.690547,16.690547,16.690547,16.690547,16.690547,16.690547
3,0.104639,0.104639,0.104639,0.104639,0.104639,0.104639
4,34.891229,34.891229,34.891229,34.891229,34.891229,34.891229
5,0.329376,0.329376,0.329376,0.329376,0.329376,0.329376
6,10.802214,10.802214,10.802214,10.802214,10.802214,10.802214
7,31.707462,31.707462,31.707462,31.707462,31.707462,31.707462
8,4.220051,4.220051,4.220051,4.220051,4.220051,4.220051


Medidas estadisticas por ultimo nivel educativo alcanzado

In [32]:
descripcion_educacion_c1 = comuna1_poblacion["P09"].describe()
descripcion_educacion_c1

count    168197.000000
mean          4.971409
std           1.975536
min           0.000000
25%           4.000000
50%           4.000000
75%           7.000000
max           9.000000
Name: P09, dtype: float64

moda de la condiciíon de actividad

In [33]:
moda_poblacion_actividad_c1 = comuna1_poblacion["CONDACT"].mode()
moda_poblacion_actividad_c1

0    1
Name: CONDACT, dtype: int64

cantidad de casos por condiciíon de actividad

In [34]:
poblacion_actividad_c1 = comuna1_poblacion.groupby(['CONDACT']).count()
poblacion_actividad_c1

,COMUNA,P03,P05,P07,P08,P09
CONDACT,,,,,,
0,15274,15274,15274,15274,15274,15274
1,113509,113509,113509,113509,113509,113509
2,5281,5281,5281,5281,5281,5281
3,34133,34133,34133,34133,34133,34133


porcentaje por condiciíon de actividad

In [35]:
porcentaje_actividad_c1 = poblacion_actividad_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_actividad_c1

,COMUNA,P03,P05,P07,P08,P09
CONDACT,,,,,,
0,9.081018,9.081018,9.081018,9.081018,9.081018,9.081018
1,67.485746,67.485746,67.485746,67.485746,67.485746,67.485746
2,3.139771,3.139771,3.139771,3.139771,3.139771,3.139771
3,20.293465,20.293465,20.293465,20.293465,20.293465,20.293465


VARIABLES DE VIVIENDA

creamos un dataframe que nos muestra las variables de vivienda para la comuna 1

In [ ]:
comuna1_vivienda = comuna1[["COMUNA", "TIPVV", "V01", "INMAT", "INCALSERV", "INCALCONS"]].copy()
comuna1_vivienda

MODA de tipo de tipo de vivienda (particular o colectiva)

In [37]:
vivienda_agrupado_c1 = comuna1_vivienda["TIPVV"].mode()
vivienda_agrupado_c1

0    1
Name: TIPVV, dtype: int64

MODA de tipo de tipo de vivienda (V01)

In [ ]:
tipo_vivienda_c1 = comuna1_vivienda["V01"].mode()
tipo_vivienda_c1

cantidad por tipo de vivienda

In [ ]:
cant_tipo_vivienda_c1 = comuna1_vivienda.groupby(['V01']).count()
cant_tipo_vivienda_c1

porcentaje de tipo de vivienda

In [ ]:
porcentaje_tipo_vivienda_c1 = cant_tipo_vivienda_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_tipo_vivienda_c1

moda de calidad de materiales de la vivienda

In [ ]:
moda_calidad_materiales_c1 = comuna1_vivienda["INMAT"].mode()
moda_calidad_materiales_c1

personas en viviendas de la comuna 1 agrupadas por indice de calidades materiales

In [ ]:
calidad_materiales_c1 = comuna1_vivienda.groupby(['INMAT']).count()
calidad_materiales_c1

porcentaje de personas en viviendas de la comuna 1 agrupadas por indice de calidades materiales

In [ ]:
porcentaje_calidad_materiales_c1 = calidad_materiales_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_calidad_materiales_c1

personas en viviendas de la comuna 1 agrupadas por indice de calidad de servicios

In [ ]:
calidad_serv_c1 = comuna1_vivienda.groupby(['INCALSERV']).count()
calidad_serv_c1

porcentaje de personas en viviendas de la comuna 1 agrupadas por indice de calidad de servicios

In [ ]:
porcentaje_calidad_serv_c1 = calidad_serv_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_calidad_serv_c1

personas en viviendas de la comuna 1 agrupadas por indice de calidad de construcción

In [ ]:
calidad_cons_c1 = comuna1_vivienda.groupby(['INCALCONS']).count()
calidad_cons_c1

porcentaje de personas en viviendas de la comuna 1 agrupadas por indice de calidad de construcción

In [ ]:
porcentaje_calidad_cons_c1 = calidad_cons_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_calidad_cons_c1

VARIABLES DE HOGAR

creamos un dataframe que nos muestra las variables de hogar para la comuna 1

In [39]:
comuna1_hogar = comuna1[["COMUNA", "H05", "H06", "H07", "H08", "H09", "H10", "H13", "H16", "H19A", "H19B", "H19C", "PROP", "INDHAC"]].copy()
comuna1_hogar

,COMUNA,H05,H06,H07,H08,H09,H10,H13,H16,H19A,H19B,H19C,PROP,INDHAC
0,1,1,4,1,1,1,1,1,2,1,2,1,5,1
1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
2,1,1,3,1,1,1,1,1,5,1,1,1,1,1
3,1,1,3,1,1,1,1,1,5,1,1,1,1,1
4,1,1,1,1,1,1,1,1,5,1,1,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205881,1,0,0,0,0,0,0,0,0,0,0,0,0,7
205882,1,0,0,0,0,0,0,0,0,0,0,0,0,7
205883,1,0,0,0,0,0,0,0,0,0,0,0,0,7
205884,1,0,0,0,0,0,0,0,0,0,0,0,0,7


personas en hogares agrupados por acceso al agua potable

In [40]:
hogar_agua_c1 = comuna1_hogar.groupby(['H08']).count()
hogar_agua_c1

,COMUNA,H05,H06,H07,H09,H10,H13,H16,H19A,H19B,H19C,PROP,INDHAC
H08,,,,,,,,,,,,,
0,15526,15526,15526,15526,15526,15526,15526,15526,15526,15526,15526,15526,15526
1,142071,142071,142071,142071,142071,142071,142071,142071,142071,142071,142071,142071,142071
2,10203,10203,10203,10203,10203,10203,10203,10203,10203,10203,10203,10203,10203
3,397,397,397,397,397,397,397,397,397,397,397,397,397


porcentaje de personas en hogares agrupados por acceso al agua potable

In [ ]:
porcentaje_agua_c1 = hogar_agua_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_agua_c1

personas en hogares agrupados por acceso a baño

In [ ]:
hogar_bagno_c1 = comuna1_hogar.groupby(['H10']).count()
hogar_bagno_c1

porcentaje de personas en hogares agrupados por acceso a baño

In [ ]:
porcentaje_bagno_c1 = hogar_bagno_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_bagno_c1

In [ ]:
hogar_bagno_ex_c1 = comuna1_hogar.groupby(['H13']).count()
hogar_bagno_ex_c1

In [ ]:
porcentaje_bagno_ex_c1 = hogar_bagno_ex_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_bagno_ex_c1

personas en hogares propios

In [ ]:
hogar_prop_c1 = comuna1_hogar.groupby(['PROP']).count()
hogar_prop_c1

porcentaje de personas en hogares propios

In [ ]:
porcentaje_hogar_prop_c1 = hogar_prop_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_hogar_prop_c1

personas en hogares agrupadasa por indice de hacinamiento

In [ ]:
hogar_hac_c1 = comuna1_hogar.groupby(['INDHAC']).count()
hogar_hac_c1

porcentaje de personas en hogares agrupadasa por indice de hacinamiento

In [ ]:
porcentaje_hogar_hac_c1 = hogar_hac_c1.transform(
    (lambda x: (x + 0.0) / 168197 * 100), axis=1
)
porcentaje_hogar_hac_c1